# langchain 中的Agents如何实现
- 几种内置的主要agent类型
    - OPENAI_FUNCTIONS Openai函数调用型
    - ZERO_SHOT_REACT_DESCRIPTION 零样本增强
    - CHAT_ZERO_SHOT_REACT_DESCRIPTION 零样本增强生成对话
    - CONVERSATION_REACT_DESCRIPTION 对话增强生成型
    - STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION 结构化对话生成增强型


ZERO_SHOT_REACT_DESCRIPTION 零样本增强式生成，即在没有示例的情况下可以自主的进行对话的类型。https://blog.csdn.net/zcyzcyjava/article/details/127006287 [零样本、单样本、少样本]

In [1]:
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv()

import pandas as pd
pd.set_option('display.max_row',300)

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.template)
agent.invoke("现在美国总统是谁？他的年龄除以2是多少？")

c:\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


verbose=True tags=['zero-shot-react-description'] agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'), llm=ChatOpenAI(client=<openai.resources.

{'input': '现在美国总统是谁？他的年龄除以2是多少？', 'output': '40.5'}

# CHAT_ZERO_SHOT_REACT_DESCRIPTION 使用了chatmodel

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
agent.invoke("现在美国总统是谁？他的年龄除以2是多少？")

verbose=True tags=['chat-zero-shot-react-description'] agent=ChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: Search, Calculator\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{{\n  "action": $TOOL_N

{'input': '现在美国总统是谁？他的年龄除以2是多少？',
 'output': 'Joe Biden is the current President of the United States and his age divided by 2 is 40.5.'}

# CONVERSATIONAL_REACT_DESCRIPTION 一个对话型的agent，这个agent要求与memory一起使用

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.template)

memory=ConversationBufferMemory(memory_key='chat_history') verbose=True tags=['conversational-react-description'] agent=ConversationalAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Ass

In [5]:
agent.run("hi i am tomie")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hello Tomie! How can I assist you today?

> Finished chain.


'Hello Tomie! How can I assist you today?'

In [6]:
agent.run("what is my name?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Your name is Tomie.

> Finished chain.


'Your name is Tomie.'

In [8]:
agent.run("有什么好吃的泰国菜可以推荐给我吗?中文输出")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: Delicious Thai dishes recommendations in Chinese
Observation: ['Top 10 Chinese & Thai Dishes · 001 BBQ Pork · 002. Fried Calamari · 003. Fried Shrimps Jumbo · 004. Fried Won-Ton-Chips · 017 Thai Fried Chicken Wings · 026. Thai ...', '10 Best Thai Food Dishes You Must Eat · 1. Pad Thai (Stir-Fried Noodles) · 2. Tom Yum Goong (Hot & Sour Shrimp Soup) · 3. Kaeng Lueang (Yellow ...', 'I would recommend - spicy wing bean salad with prawn (personal favourite), Penang curry, Chuchi curry with river prawn and fried rice with sour ...', "5 Thai-Chinese dishes to try in Bangkok's Yaowarat district · Fishballs · Khao Moo Daeng · Jok · Guay Jub · Yentafo · Latest Stories · Footer · Privacy ...", '1. **Pad Thai**: A stir-fried noodle dish, typically made with rice noodles, shrimp or chicken, peanuts, a scrambled egg, and bean sprouts, ...', "Noodles, soy sauce, chicken and broccoli. That's it.

'Here are some delicious Thai dishes that I can recommend to you in Chinese:\n1. 泰式炒河粉 (Pad Thai)\n2. 冬阴功汤 (Tom Yum Goong)\n3. 黄咖喱 (Kaeng Lueang)\n4. 红咖喱 (Red Curry)\n5. 绿咖喱 (Green Curry)\n6. 泰式炒肉碎 (Pad Kra Pao Moo)\n7. 泰式青木瓜沙拉配烤鸡 (Som Tom with Grilled Chicken)\n8. 辣味鸡肉香肠 (Laab Gai & Laab Moo)\n9. 泰式香肠 (Sai Ua)\n10. 冬阴汤 (Tom Yum)\nEnjoy your Thai food adventure!'

In [9]:
agent.run("这些我都没吃过！我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: "1998 World Cup winner"
Observation: France national football team
Thought:Do I need to use a tool? Yes
Action: Search
Action Input: "What is the last letter of my name?"
Observation: ['Our surname begins with an F (think Fox), and the names combined are a bit of a tongue twister. My family says it sounds fine, but I worry the ...', 'Give your room the glow up it deserves with colorful LED lighting! https://glowupleds.com/?ref=tiktokeri 10% Off with Code "TTC" Free ...', 'Several of my favorite boy names end in x and my last name starts with s. I love the names so much (particularly [name]Hendrix[/name]!) that I ...', 'Hi, Last letter of my username under C:\\Users is missing. It was missing from my previous PC. I just got a new PC this week, logged in with ...', 'The Capstone – The Capstone is the last letter of your name, and, fittingly, it demonstrates your ability to see proje

'The last letter of your name is "e". The winner of the 1998 World Cup was the France national football team.'

In [10]:
agent.run("中国陕西西安现在的气温多少？截止目前我们聊了什么？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: Current temperature in Xi'an, China
Observation: {'type': 'weather_result', 'temperature': '86', 'unit': 'Fahrenheit', 'precipitation': '32%', 'humidity': '38%', 'wind': '7 mph', 'location': "Xi'An, Shaanxi, China", 'date': 'Friday 5:00 PM', 'weather': 'Mostly sunny'}
Thought:Do I need to use a tool? No
AI: The current temperature in Xi'an, China is 86 degrees Fahrenheit. As for our conversation, we discussed Thai cuisine recommendations, the last letter of your name, the winner of the 1998 World Cup, and the current weather in Xi'an.

> Finished chain.


"The current temperature in Xi'an, China is 86 degrees Fahrenheit. As for our conversation, we discussed Thai cuisine recommendations, the last letter of your name, the winner of the 1998 World Cup, and the current weather in Xi'an."

In [18]:
# CHAT_CONVERSATIONAL_REACT_DESCRIPTION 使用了chatmodel
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
print(agent.agent.llm_chain.prompt.messages[2].prompt.template)
print(agent.agent.llm_chain.prompt.messages[3])

memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['chat-conversational-react-description'] agent=ConversationalChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Ass

In [19]:
agent.run(input="hi i am tony")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Hello Tony! How can I assist you today?"
}
```

> Finished chain.


'Hello Tony! How can I assist you today?'

In [20]:
agent.run(input="what is my name?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Your name is Tony."
}
```

> Finished chain.


'Your name is Tony.'

In [21]:
agent.run(input="有什么好吃的泰国菜给我推荐一下？")



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "Popular Thai dishes to try"
}
```
Observation: To help you, here is a list of the top ten most popular Thai dishes that you absolutely have to try.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Here is a list of the top ten most popular Thai dishes that you absolutely have to try."
}
```

> Finished chain.


'Here is a list of the top ten most popular Thai dishes that you absolutely have to try.'

In [22]:
agent.run(input="我都没吃过！我名字的最后一个字母是什么？1998年的世界杯谁夺冠了？")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "y"
}
```

> Finished chain.


'y'

# OPENAI_FUNCTIONS，使用openai的函数调用来实现的，只支持openai模型。

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory,#记忆组件
    verbose=True,
)
print(agent)
print(agent.agent.prompt.messages)

memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['openai-functions'] agent=OpenAIFunctionsAgent(llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000019497F7CE30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000019497F7E120>, temperature=0.0, openai_api_key='fk226347-RSOUezCLbQHfanhNJZsQl6Zx9nkKBMpo', openai_api_base='https://openai.api2d.net', openai_proxy=''), tools=[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='4ba6f3d3a1010b2252ca53ebe8fc77c1727eadbc894149a03a5fb5039d86194c', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWra

In [29]:
agent.run(input="今天比特币的价格是多少？它的价格开平方是多少？")



> Entering new AgentExecutor chain...

Invoking: `Search` with `Bitcoin price today`


62,945.60
Invoking: `Calculator` with `sqrt(62945.60)`


Answer: 250.88961716260798今天比特币的价格是62,945.60美元，它的价格开平方是约250.89美元。

> Finished chain.


'今天比特币的价格是62,945.60美元，它的价格开平方是约250.89美元。'

# STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION 对输出做了结构化处理

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType,
)
from langchain.memory import ConversationBufferMemory
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,#记忆组件
    handle_parsing_errors=True,
    verbose=True,
)
print(agent)
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)
print(agent.agent.llm_chain.prompt.messages[1].prompt.template)

memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True tags=['structured-chat-zero-shot-react-description'] agent=StructuredChatAgent(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args: {{\'tool_input\': {{\'type\': \'string\'}}}}\nCalculator: Useful for when you need to answer questions about math., args: {{\'tool_input\': {{\'type\': \'string\'}}}}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or Search, Calculator\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n

In [31]:
agent.invoke({"input":"what is langchain?"})



> Entering new AgentExecutor chain...
I'm not familiar with "langchain." Would you like me to look it up for you?


> Finished chain.


{'input': 'what is langchain?',
 'chat_history': [HumanMessage(content='what is langchain?'),
  AIMessage(content='I\'m not familiar with "langchain." Would you like me to look it up for you?\n')],
 'output': 'I\'m not familiar with "langchain." Would you like me to look it up for you?\n'}